In [1]:
# Import Libraries
import pymongo
import json
from bson.objectid import ObjectId
from datetime import datetime
import pandas as pd

In [2]:

#Establish the connection and make the database
client = pymongo.MongoClient("mongodb://localhost:27017/") 
db_list = client.list_database_names()

In [3]:
db_list


['Tweets', 'admin', 'config', 'local', 'mydatabase']

In [4]:
if "Tweets" in db_list:
    print("Database already exists. Will be dropped...")
    client.drop_database("Tweets")
db = client["Tweets"]
col_list = db.list_collection_names()
if "Tweets_collection" in col_list:
        print("Tweets Collection already exists. Will be dropped...")
        db.Tweets_data.drop()
tweets_collec = db["Tweets_collection"]
print("Database connection established")

Database already exists. Will be dropped...
Database connection established


In [5]:
def insert_tweet(tweet_info):
    try:
        tweets_collec.insert_one(tweet_info)
    except Exception as e:
        print(str(e))

In [6]:
def find_tweet(tweet_id):
    tweet = tweets_collec.find_one({"Tweet_Id": tweet_id})
    if tweet:
        return True
    else:
        return False

In [7]:
def retweet_update(tweet_id, retweet_id):
    try:
        tweets_collec.update_one({'Tweet_Id': tweet_id}, {"$inc": {'Retweet_Count': 1}})
    except Exception as e:
        print(f"Error updating tweet {tweet_id}: {e}")

In [8]:
# function to load data

def process_tweets(file_path):
    with open(file_path, "r") as read_file:
        for line in read_file:
            try:
                data = json.loads(line)

                # check if tweet already exists
                if find_tweet(data['id_str']):
                    continue

                # check for retweet
                if data['text'].startswith('RT'):
                    if data['retweeted_status']!=None:
                        source_tweet_Id = data['retweeted_status']['id_str']
                        if find_tweet(source_tweet_Id):
                            retweet_update(source_tweet_Id, data['id_str'])
                        else:
                            tweet_info = {
                                'created_at':datetime.strftime(datetime.strptime(data['retweeted_status']['created_at'],'%a %b %d %H:%M:%S +0000 %Y'),'%Y-%m-%d %H:%M:%S'),
                                'Tweet_Id':data['retweeted_status']['id_str'],
                                'Text':data['retweeted_status']['text'],
                                'Hashtag':list(map(lambda x: x["text"], data['retweeted_status']['entities']['hashtags'])),
                                'User_Id':data['retweeted_status']['user']['id_str'],
                                'User_Name':data['retweeted_status']['user']['name'],
                                'Retweet_Count': 1,
                                'Likes_Count': data['retweeted_status']['favorite_count']
                            }
                            insert_tweet(tweet_info)
                else:
                    source_tweet_Id = 0

                retweet_info = {
                    'created_at':datetime.strftime(datetime.strptime(data['created_at'],'%a %b %d %H:%M:%S +0000 %Y'),'%Y-%m-%d %H:%M:%S'),
                    'Tweet_Id':data['id_str'],
                    'Text':data['text'],
                    'Hashtag':list(map(lambda x: x["text"], data['entities']['hashtags'])),
                    'User_Id':data['user']['id_str'],
                    'User_Name':data['user']['name'],
                    'Source_tweet_Id':source_tweet_Id,
                    'Retweet_Count': 0,
                    'Likes_Count': data['favorite_count']
                }
                insert_tweet(retweet_info)

            except:
                continue


In [9]:
 process_tweets('corona-out-3')

In [10]:
tweets_collec.count_documents({})

112014